In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
from collections import defaultdict
import json
import re
import requests
import sys
import random
import requests
import tabulate
from datetime import datetime
import schedule
import time

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
#Get work logs
sql = f'''
SELECT
  PUBLIC.QAEVENTS.user_email :: string EMAIL,
  PUBLIC.QAEVENTS.metadata: pvids PVIDS,
  (PUBLIC.QAEVENTS.metadata: totalMillisElapsed)/100 TIMESPENTSEC,
  --  PUBLIC.QAEVENTS.metadata: fieldBefore BEFORE,
  --  PUBLIC.QAEVENTS.metadata: fieldCurrent AFTER,
  PUBLIC.QAEVENTS.event_type,
  PUBLIC.QAEVENTS.event_at,
  cast (PUBLIC.QAEVENTS.event_at as DATE) AS EventDate,
  --  PUBLIC.QAEVENTS.metadata: fullStoryEvent :: string,
  --  PUBLIC.QAEVENTS.metadata: auditLevel :: string,
  --  PUBLIC.QAEVENTS.metadata: hintsAvailable,
  PUBLIC.QAEVENTS.metadata: domain SITE
FROM
  PUBLIC.QAEVENTS
WHERE
--        CAST(PUBLIC.QAEVENTS.event_at AS DATE) = CURRENT_DATE()
      CAST(PUBLIC.QAEVENTS.event_at AS DATE) BETWEEN '2022-09-27' AND '2022-10-27'
  AND PUBLIC.QAEVENTS.metadata :domain is not NULL
  AND PUBLIC.QAEVENTS.event_type = 'submit_description_speed_audit'
  AND PUBLIC.QAEVENTS.user_email NOT LIKE '%telework%'
'''
cs.execute(sql)
worklogdf = cs.fetch_pandas_all()


In [14]:
worklogdf

,EMAIL,PVIDS,TIMESPENTSEC,EVENT_TYPE,EVENT_AT,EVENTDATE,SITE,LEVEL
0,karla.nunez@contractors.scale.com,"[\n ""shop.jenessawait.com!31943155089480"",\n ...",529.56,submit_description_speed_audit,2022-09-28 23:01:18.196000+00:00,2022-09-28,"""shop.jenessawait.com""",QA
1,karla.nunez@contractors.scale.com,"[\n ""shop.jenessawait.com!39555769663643"",\n ...",107.53,submit_description_speed_audit,2022-09-28 23:01:29.045000+00:00,2022-09-28,"""shop.jenessawait.com""",QA
2,karla.nunez@contractors.scale.com,"[\n ""shop.jenessawait.com!35184148250779"",\n ...",114.75,submit_description_speed_audit,2022-09-28 23:01:57.133000+00:00,2022-09-28,"""shop.jenessawait.com""",QA
3,samantha.couoh@contractors.scale.com,"[\n ""snappyscreen.com!39388791373964""\n]",760.86,submit_description_speed_audit,2022-09-28 22:35:48.247000+00:00,2022-09-28,"""snappyscreen.com""",QA
4,karla.nunez@contractors.scale.com,"[\n ""shop.jenessawait.com!39582131290267"",\n ...",16.19,submit_description_speed_audit,2022-09-28 22:56:08.311000+00:00,2022-09-28,"""shop.jenessawait.com""",QA
...,...,...,...,...,...,...,...,...
1548090,joyce.palma@contractors.scale.com,"[\n ""inakapower.com!39679303483438"",\n ""inak...",52.88,submit_description_speed_audit,2022-10-27 23:52:09.919000+00:00,2022-10-27,"""inakapower.com""",QA
1548091,mariana.cuevas@contractors.scale.com,"[\n ""eightsaintsskincare.com!32938272981095""\n]",29.61,submit_description_speed_audit,2022-10-27 23:51:17.294000+00:00,2022-10-27,"""eightsaintsskincare.com""",QA
1548092,german.toledo@contractors.scale.com,"[\n ""slowdownstudio.com!40620005720242""\n]",3.97,submit_description_speed_audit,2022-10-27 23:51:46.542000+00:00,2022-10-27,"""slowdownstudio.com""",QA
1548093,german.toledo@contractors.scale.com,"[\n ""slowdownstudio.com!40385501593778""\n]",5.37,submit_description_speed_audit,2022-10-27 23:52:15.115000+00:00,2022-10-27,"""slowdownstudio.com""",QA


In [16]:
exportdf = worklogdf.groupby(['EMAIL','EVENTDATE'])['PVIDS'].count()


In [18]:
exportdf = exportdf.to_frame()

In [20]:
exportdf.to_csv('descriptionsperqa.csv')

In [4]:
#Create QA or Teleworker classification column, convert timespent to minutes
worklogdf['LEVEL'] = np.where(worklogdf['EMAIL'].str.contains("contractors"), 'QA', np.where(worklogdf['EMAIL'].str.contains("telework"), 'TW', 'ScaleHQ'))
#worklogdf

In [5]:
#Summarize reports
qacount = worklogdf.groupby(['EVENTDATE','LEVEL'])['EMAIL'].nunique()
#qacount

In [6]:
descount = worklogdf.groupby(['LEVEL'])['EVENTDATE'].count()
descount = descount.to_frame()
descount.columns = ['Descriptions']

In [7]:
#get descriptions count
tot = descount['Descriptions'].sum()
#tot

In [8]:
#get sites count
tmpct=[]
sitecount=0
for i in range(0, len(worklogdf['SITE'])):
    if worklogdf['SITE'][i] not in tmpct: 
        tmpct.append(worklogdf['SITE'][i])
        sitecount += 1
#print(sitecount)

In [9]:
#get description count by QA
qadescount = worklogdf.groupby(['EMAIL','LEVEL'])['EVENTDATE'].count()
qadescount = qadescount.to_frame()
qadescount = qadescount.sort_values(['EVENTDATE'], ascending=False)
#qadescount
#qadescount = qadescount.sort_values(by='Descriptions', ascending=False)
#qdlite = qadescount.drop(['EVENTDATE'], axis=1)

In [10]:
#Assumption 1 - get average descriptions per site
sitedesdf = worklogdf.groupby(['SITE'])['EVENTDATE'].count()
sddf = sitedesdf.to_frame()

#Remove outlier low description count sites
sddf.drop(sddf[sddf['EVENTDATE'] < 50].index, inplace = True)

#Calculate average of column
avg_dcount = sddf['EVENTDATE'].mean()
print("Average descriptions per site =",avg_dcount)

Average descriptions per site = 1769.7379310344827


In [11]:
#Assumption 2 - Descriptions per hour
qadesdf = worklogdf.groupby(['EVENTDATE','EMAIL','LEVEL'])['EVENTDATE'].count()
qadf = qadesdf.to_frame()
qadf.columns = ['DCOUNT']

In [12]:
#Remove outlier low work instances
qadf.drop(qadf[qadf['DCOUNT'] < 200].index, inplace = True)
taskerdf = qadf.groupby(['EMAIL','LEVEL'])['DCOUNT'].mean()
tdf = taskerdf.to_frame()

In [13]:
#Remove temp workers - agents and ScaleHQ
#tdf.drop(tdf[tdf['DCOUNT'] < 800].index, inplace = True)
print("Average descriptions per hour by worker type")
fdf = tdf.groupby(['LEVEL'])['DCOUNT'].mean()
adf = fdf.to_frame()
adf['DCOUNT'] = adf['DCOUNT'].div(8).round(2)
adf

Average descriptions per hour by worker type


,DCOUNT
LEVEL,
QA,440.21
ScaleHQ,181.39
